# Exercice 4: Attaquer le système en modifiant directement le fichier de données, en changeant le montant d’une transaction.


In [1]:
import os
current_dir = os.getcwd()
new_dir = current_dir.replace("\\tests", "")
os.chdir(new_dir)
from services.transaction_service import TransactionService
from services.person_service import PersonService

In [3]:
# Init

person_service = PersonService()
transaction_service = TransactionService()


if person_service.is_persons_empty():
    person_service.add_person("John", "Doe", "1990-01-01", 1500.0)
    person_service.add_person("Jane", "Dupont", "1990-01-01", 780.0)
if transaction_service.is_transactions_empty():
    if not transaction_service.add_transaction(1, 2, 150.0):
        raise Exception("Transaction failed")

In [4]:
# Old transaction value
transaction_service = TransactionService()
transaction = transaction_service.get_transaction(1)
print(transaction)

 1dd3c39cdbfe268ab759d8f24d537471f27a8b1853451c42407f644db9bc1d78 2023-12-04 16:30:10.604911 - (id: 1) John Doe 1350.0€ -> (id: 2) Jane Dupont 930.0€: 150.0€


In [5]:
# Update transaction amount
transaction.amount = 1000
transaction_service.update_transaction(transaction)

In [6]:
# New transaction value
transaction = transaction_service.get_transaction(1)
print(transaction)

Exception: Hashes do not match

# Exercice 7. Vérifiez que l’attaque précédente ne fonctionne plus.

In [4]:
import os
current_dir = os.getcwd()
new_dir = current_dir.replace("\\tests", "")
os.chdir(new_dir)
from services.transaction_service import TransactionService
from services.person_service import PersonService

In [5]:
# Init

person_service = PersonService()
transaction_service = TransactionService()

if person_service.is_persons_empty():
    person_service.add_person("John", "Doe", "1990-01-01", 1500.0)
    person_service.add_person("Jane", "Dupont", "1990-01-01", 780.0)
if transaction_service.is_transactions_empty():
    if not transaction_service.add_transaction(1, 2, 150.0):
        raise Exception("Transaction failed")

In [6]:
# Old transaction value
transaction_service = TransactionService()
transaction = transaction_service.get_transaction(1)
print(transaction)

 dcb6987a517d03c408c49436e1110b30ee88038b727552a9bb4195add1494cfd 2023-12-05 11:33:08.248597 - (id: 1) John Doe 1350.0€ -> (id: 2) Jane Dupont 930.0€: 150.0€


In [10]:
# Update transaction amount
transaction.amount = 1000
transaction_service.update_transaction(transaction)

{'res': 'Error during hash verification for transaction ID: 1\nError: Hashes do not match\n', 'count': 1}


In [8]:
# New transaction amount
transaction = transaction_service.get_transaction(1)
print(transaction)

Exception: Hashes do not match

In [9]:
print(transaction_service.check_hash_transactions())

{'res': 'Error during hash verification for transaction ID: 1\nError: Hashes do not match\n', 'count': 1}


# Exercice 8: Attaquer le système en modifiant directement le fichier de données, en supprimant unetransaction. La possibilité de supprimer une transaction peut être très dangereuse, la suppression peut entraîner la double dépense [9]

In [1]:
import os
current_dir = os.getcwd()
new_dir = current_dir.replace("\\tests", "")
os.chdir(new_dir)
from services.transaction_service import TransactionService
from services.person_service import PersonService

In [2]:
# Init

person_service = PersonService()
transaction_service = TransactionService()


if person_service.is_persons_empty():
    person_service.add_person("John", "Doe", "1990-01-01", 1500.0)
    person_service.add_person("Jane", "Dupont", "1990-01-01", 780.0)
if transaction_service.is_transactions_empty():
    if not transaction_service.add_transaction(1, 2, 150.0):
        raise Exception("Transaction failed")
    if not transaction_service.add_transaction(1, 2, 50.0):
        raise Exception("Transaction failed")
    if not transaction_service.add_transaction(2, 1, 314.99):
        raise Exception("Transaction failed")

In [3]:
transaction_service.delete_transaction(2)

In [4]:
print(transaction_service.check_hash_transactions())

{'res': 'Transaction ID: 1 is valid\nTransaction ID: 3 is valid\n', 'count': 0}


# Exercice 10: Vérifiez que les attaques précédentes ne fonctionnent plus.

In [4]:
print(transaction_service.check_hash_transactions())

{'res': 'Transaction ID: 1 is valid\nTransaction ID: 3 is invalid. Error during hash verification.\n', 'count': 1}


#  Exercice 11: Attaquer le système en modifiant directement le fichier de données, en ajoutant, par exemple, une transaction provenant d’une autre personne vers le compte de l’attaquant.

In [1]:
import os
current_dir = os.getcwd()
new_dir = current_dir.replace("\\tests", "")
os.chdir(new_dir)
from services.transaction_service import TransactionService
from services.person_service import PersonService

In [2]:
person_service = PersonService()
transaction_service = TransactionService()

if person_service.is_persons_empty():
    person_service.add_person("John", "Doe", "1990-01-01", 1500.0)
    person_service.add_person("Jane", "Dupont", "1990-01-01", 780.0)

# Attempting to add a transaction from a person (ID: 2) to the attacker's account (ID: 1)
if not transaction_service.add_transaction(2, 1, 100.0):
    print("Transaction failed - Person not found.")
else:
    print("Transaction added successfully.")

print(transaction_service.check_hash_transactions())

Transaction added successfully.
{'res': 'Transaction ID: 1 is valid\n', 'count': 0}


#  Exercice 13: Preuve de l'impossibilité d'usurper une identité.

In [1]:
import os
current_dir = os.getcwd()
new_dir = current_dir.replace("\\tests", "")
os.chdir(new_dir)
from services.transaction_service import TransactionService
from services.person_service import PersonService

person_service = PersonService()
transaction_service = TransactionService()

In [2]:
victim, victim_private_key = person_service.add_person("John", "Doe", "1990-01-01", 1500.0)
attacker, attacker_private_key = person_service.add_person("Mr", "Robot", "1990-01-01", 0.0)

In [3]:
print(victim)
print(attacker)

(id: 4) John Doe 1500.0€
(id: 5) Mr Robot 0.0€


In [6]:
# Assuming the attacker wants to create a transaction which will takes 1000 from the true true user's account
transaction_data = {
    "p1_id": victim.id,
    "p2_id": attacker.id,
    "amount": 1000.0,
}

# Attacker does not have the victim's private key, so he signs using his own private key
attacker_signature = transaction_service.sign_transaction(attacker_private_key, transaction_data)

transaction_service.add_transaction(transaction_data, attacker_signature)

Invalid request: Invalid signature


False

In [7]:
# The attacker can't send any money to himself using the victim's account.
# Now if the victim wants to send money to the attacker he can.
# Assuming the victim wants to send 1000 to the attacker

transaction_data = {
    "p1_id": victim.id,
    "p2_id": attacker.id,
    "amount": 1000.0,
}

# Victim signs the transaction using his private key
victim_signature = transaction_service.sign_transaction(victim_private_key, transaction_data)
transaction_service.add_transaction(transaction_data, victim_signature)

True